#Intro
    

yt-dlp from within colab to local disk
then output to gdrive, mega, onedrive?

In [ ]:
# Install yt-dlp (quietly to suppress verbose output)
!pip install yt-dlp
!apt-get install -y ffmpeg

##Download With YT-DLP
saves to file `downloaded_video.mp4`


In [ ]:
# Import necessary libraries
from google.colab import files

# Function to download a video using yt-dlp
def download_video(url, output_path="downloaded_video"):
    """
    Downloads a video from a URL using yt-dlp.

    Args:
        url (str): The URL of the video to download.
        output_path (str): The base name of the output file.
    """
    from yt_dlp import YoutubeDL

    # Set options for yt-dlp
    ydl_opts = {
        'outtmpl': f'{output_path}.%(ext)s',  # Output file template
        'format': 'bestvideo+bestaudio/best', # Download best quality video and audio
        'merge_output_format': 'mp4'         # Merge video and audio into mp4
    }

    # Use yt-dlp to download the video
    with YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

# Example Usage:
video_url = input("Enter the video URL: ")  # Prompt user to input the video URL
download_video(video_url)                  # Call the function to download the video

# Download the file to your local machine
#downloaded_file = "downloaded_video.mp4"
#files.download(downloaded_file)


##ConvertVideoToAudio

In [ ]:
# Define file paths
input_video = "downloaded_video.mp4"   # Input video file
output_audio = "output_audio.mp3"     # Desired output audio file

# Use FFmpeg to extract audio from the video
!ffmpeg -i "{input_video}" -vn -acodec libmp3lame -q:a 2 "{output_audio}"

# (Optional) Download the audio file to your local machine (uncomment to enable)
#from google.colab import files
#files.download(output_audio)

In [ ]:
# Play the audio file from prior code in Colab
from IPython.display import Audio
Audio(output_audio)

# Play a specific audio file only
#output_audio = "output_audio.mp3"
#Audio(output_audio, autoplay=True)

###PlayingAudioFiles

In [ ]:
from IPython.display import Audio
import os

# List all audio files in the current directory
audio_files = [f for f in os.listdir() if f.endswith(('.mp3', '.wav'))]

# Display available audio files
print("Available audio files:")
for i, audio_file in enumerate(audio_files):
    print(f"{i + 1}: {audio_file}")

# Let the user choose a file to play
choice = int(input("Enter the number of the audio file you want to play: ")) - 1

# Play the selected audio file
Audio(audio_files[choice], autoplay=True)


In [ ]:
import os
print(os.listdir('/content'))


In [ ]:
from yt_dlp import YoutubeDL

# Function to get available formats for a video using yt-dlp
def get_available_formats(url):
    ydl_opts = {
        'quiet': True,  # Suppress unnecessary output
        'extract_flat': True,  # Only extract metadata (no download)
    }

    with YoutubeDL(ydl_opts) as ydl:
        info_dict = ydl.extract_info(url, download=False)  # Extract metadata without downloading
        formats = info_dict.get('formats', [])
        return formats

# Function to download a video using yt-dlp based on selected quality
def download_video(url, quality_option=1, output_path="downloaded_video"):
    """
    Downloads a video from a URL using yt-dlp with selected quality.

    Args:
        url (str): The URL of the video to download.
        quality_option (int): The quality preference for video download (1=720p, 2=1080p, 3=4K or higher).
        output_path (str): The base name of the output file.
    """
    # Get the available formats for the video
    available_formats = get_available_formats(url)

    # Define the quality options based on the user's selection
    if quality_option == 1:  # 720p
        preferred_formats = [f for f in available_formats if f.get('height') and f['height'] <= 720]
        fallback_quality = 2  # Fallback to 1080p
    elif quality_option == 2:  # 1080p
        preferred_formats = [f for f in available_formats if f.get('height') and f['height'] <= 1080]
        fallback_quality = None
    elif quality_option == 3:  # 4K or higher
        preferred_formats = [f for f in available_formats if f.get('height') and f['height'] >= 2160]
        fallback_quality = 2  # Fallback to 1080p
    else:
        print("Invalid selection. Defaulting to 720p.")
        preferred_formats = [f for f in available_formats if f.get('height') and f['height'] <= 720]
        fallback_quality = 2  # Fallback to 1080p

    # Check if the preferred format is available
    if preferred_formats:
        # Select the best available preferred format (e.g., highest quality in the list)
        best_format = max(preferred_formats, key=lambda x: x.get('height', 0))
        format_id = best_format['format_id']
        print(f"Selected format: {best_format['height']}p")
    else:
        print(f"Desired quality not available. Retrying with fallback quality {fallback_quality}...")
        if fallback_quality == 2:  # Fallback to 1080p
            preferred_formats = [f for f in available_formats if f.get('height') and f['height'] <= 1080]
            if preferred_formats:
                best_format = max(preferred_formats, key=lambda x: x.get('height', 0))
                format_id = best_format['format_id']
                print(f"Selected fallback format: {best_format['height']}p")
            else:
                print("No suitable formats found.")
                return
        else:
            print("No suitable formats found for the requested quality.")
            return

    # Now, download the video with the selected format
    ydl_opts = {
        'outtmpl': f'{output_path}.%(ext)s',  # Output file template
        'format': format_id,                 # Video quality format based on selected format
        'merge_output_format': 'mp4',         # Merge video and audio into mp4
    }

    with YoutubeDL(ydl_opts) as ydl:
        try:
            ydl.download([url])  # Attempt to download using the selected format
            print("Download successful!")
        except Exception as e:
            print(f"Error downloading with format {format_id}: {e}")

# Example Usage:
video_url = input("Enter the video URL: ")  # Prompt user to input the video URL

# Ask the user to select the quality option by number
print("Select the video quality:")
print("1. 720p")
print("2. 1080p")
print("3. 4K or higher")
quality_choice = int(input("Enter your choice (1, 2, or 3): "))

# Call the function to download the video with the selected quality
download_video(video_url, quality_option=quality_choice)


In [ ]:
from yt_dlp import YoutubeDL

# Function to get available formats for a video using yt-dlp
def get_available_formats(url):
    ydl_opts = {
        'quiet': True,  # Suppress unnecessary output
        'extract_flat': True,  # Only extract metadata (no download)
    }

    with YoutubeDL(ydl_opts) as ydl:
        info_dict = ydl.extract_info(url, download=False)  # Extract metadata without downloading
        formats = info_dict.get('formats', [])
        return formats

# Function to display available video quality options
def get_video_quality_options(formats):
    quality_options = {}

    # Check available resolutions and add them to options
    for f in formats:
        height = f.get('height')
        if height:
            if height not in quality_options:
                quality_options[height] = []

            # Collect format ids for each resolution
            quality_options[height].append(f)

    # Return available options sorted by resolution
    return sorted(quality_options.items(), key=lambda x: x[0], reverse=True)

# Function to download a video using yt-dlp based on selected quality
def download_video(url, selected_quality, output_path="downloaded_video"):
    """
    Downloads a video from a URL using yt-dlp with selected quality.

    Args:
        url (str): The URL of the video to download.
        selected_quality (int): The height (resolution) to download.
        output_path (str): The base name of the output file.
    """
    # Get the available formats for the video
    available_formats = get_available_formats(url)

    # Find the selected quality format(s)
    selected_formats = [f for f in available_formats if f.get('height') == selected_quality]

    if selected_formats:
        # Select the best available selected format (e.g., highest quality in the list)
        best_format = max(selected_formats, key=lambda x: x.get('height', 0))
        format_id = best_format['format_id']
        print(f"Selected format: {best_format['height']}p")
    else:
        print(f"Quality {selected_quality}p not available. No suitable formats found.")
        return

    # Now, download the video with the selected format
    ydl_opts = {
        'outtmpl': f'{output_path}.%(ext)s',  # Output file template
        'format': format_id,                 # Video quality format based on selected format
        'merge_output_format': 'mp4',         # Merge video and audio into mp4
    }

    with YoutubeDL(ydl_opts) as ydl:
        try:
            ydl.download([url])  # Attempt to download using the selected format
            print("Download successful!")
        except Exception as e:
            print(f"Error downloading with format {format_id}: {e}")

# Example Usage:
video_url = input("Enter the video URL: ")  # Prompt user to input the video URL

# Get available formats for the provided URL
available_formats = get_available_formats(video_url)

# Get and display available quality options to the user
quality_options = get_video_quality_options(available_formats)

# Display the available resolution options
print("Select the video quality from the available options:")

quality_map = {}
for i, (height, formats) in enumerate(quality_options, start=1):
    print(f"{i}. {height}p")
    quality_map[i] = height

# Ask the user to select the quality option
quality_choice = int(input("Enter your choice (1, 2, or 3): "))

# Get the selected quality resolution (height)
selected_quality = quality_map.get(quality_choice)

if selected_quality:
    # Call the function to download the video with the selected quality
    download_video(video_url, selected_quality)
else:
    print("Invalid selection. Exiting.")


In [ ]:
## Smart Detection


import os
from yt_dlp import YoutubeDL

# Function to get available formats for a video using yt-dlp
def get_available_formats(url):
    ydl_opts = {
        'quiet': True,  # Suppress unnecessary output
        'extract_flat': True,  # Only extract metadata (no download)
    }

    with YoutubeDL(ydl_opts) as ydl:
        info_dict = ydl.extract_info(url, download=False)  # Extract metadata without downloading
        formats = info_dict.get('formats', [])
        return formats

# Function to display available video quality options
def get_video_quality_options(formats):
    quality_options = {}

    # Check available resolutions and add them to options
    for f in formats:
        height = f.get('height')
        if height:
            if height not in quality_options:
                quality_options[height] = []

            # Collect format ids for each resolution
            quality_options[height].append(f)

    # Return available options sorted by resolution
    return sorted(quality_options.items(), key=lambda x: x[0], reverse=True)

# Function to check and generate a unique file name if the file already exists
def generate_unique_filename(base_filename):
    counter = 2
    new_filename = f"{base_filename}_{counter}.mp4"

    # Check if the file already exists and increment the counter until a unique file name is found
    while os.path.exists(new_filename):
        counter += 1
        new_filename = f"{base_filename}_{counter}.mp4"

    return new_filename

# Function to download a video using yt-dlp based on selected quality
def download_video(url, selected_quality, output_path="downloaded_video"):
    """
    Downloads a video from a URL using yt-dlp with selected quality.

    Args:
        url (str): The URL of the video to download.
        selected_quality (int): The height (resolution) to download.
        output_path (str): The base name of the output file.
    """
    # Get the available formats for the video
    available_formats = get_available_formats(url)

    # Find the selected quality format(s)
    selected_formats = [f for f in available_formats if f.get('height') == selected_quality]

    if selected_formats:
        # Select the best available selected format (e.g., highest quality in the list)
        best_format = max(selected_formats, key=lambda x: x.get('height', 0))
        format_id = best_format['format_id']
        print(f"Selected format: {best_format['height']}p")
    else:
        print(f"Quality {selected_quality}p not available. No suitable formats found.")
        return

    # Generate a unique filename if the file already exists
    output_file = generate_unique_filename(output_path)

    # Now, download the video with the selected format
    ydl_opts = {
        'outtmpl': f'{output_file}.%(ext)s',  # Output file template
        'format': format_id,                 # Video quality format based on selected format
        'merge_output_format': 'mp4',         # Merge video and audio into mp4
    }

    with YoutubeDL(ydl_opts) as ydl:
        try:
            ydl.download([url])  # Attempt to download using the selected format
            print(f"Download successful! File saved as {output_file}.mp4")
        except Exception as e:
            print(f"Error downloading with format {format_id}: {e}")

# Example Usage:
video_url = input("Enter the video URL: ")  # Prompt user to input the video URL

# Get available formats for the provided URL
available_formats = get_available_formats(video_url)

# Get and display available quality options to the user
quality_options = get_video_quality_options(available_formats)

# Display the available resolution options
print("Select the video quality from the available options:")

quality_map = {}
for i, (height, formats) in enumerate(quality_options, start=1):
    print(f"{i}. {height}p")
    quality_map[i] = height

# Ask the user to select the quality option
quality_choice = int(input("Enter your choice (1, 2, or 3): "))

# Get the selected quality resolution (height)
selected_quality = quality_map.get(quality_choice)

if selected_quality:
    # Call the function to download the video with the selected quality
    download_video(video_url, selected_quality)
else:
    print("Invalid selection. Exiting.")


## 720p/1080p/>1080p quiet output


Confirmed Working.


*   Asks for url
*   Detects abvailable video qualities
*   Asks user which to download
*   Downloads file to disk (Always download locally) `downloaded_video.mp4` <br>
     Then use export code to download to local disk

In [ ]:
import os
from yt_dlp import YoutubeDL
import traceback

def get_available_formats(url):
    """
    Extract available video formats using yt-dlp's Python API with enhanced debugging.
    Returns a list of formats with high-resolution video streams.
    """
    ydl_opts = {
        'quiet': False,
        'no_warnings': False,
        'extract_flat': False
    }

    with YoutubeDL(ydl_opts) as ydl:
        try:
            print("Attempting to extract video information...")
            info = ydl.extract_info(url, download=False)

            if not info:
                print("No video information retrieved")
                return []

            formats = []
            print(f"\nTotal formats found: {len(info.get('formats', []))}")

            for f in info['formats']:
                # Modified condition: Include formats that have video and are 720p or higher
                if (f.get('vcodec') != 'none' and
                    f.get('height', 0) >= 720):

                    formats.append({
                        'format_id': f['format_id'],
                        'resolution': f.get('height'),
                        'vcodec': f['vcodec'],
                        'acodec': f.get('acodec'),
                        'filesize': f.get('filesize', 'Unknown'),
                        'format_note': f.get('format_note', '')
                    })

            print(f"\nUsable formats found (720p and above): {len(formats)}")
            return formats

        except Exception as e:
            print(f"\nError occurred while extracting format information:")
            print(f"Error message: {str(e)}")
            return []
def get_video_quality_options(formats):
    """
    Group available formats by resolution and sort them in descending order.
    """
    quality_options = {}
    for f in formats:
        resolution = f.get('resolution')
        if resolution not in quality_options:
            quality_options[resolution] = []
        quality_options[resolution].append(f)

    return sorted(quality_options.items(), key=lambda x: x[0], reverse=True)

def generate_unique_filename(base_filename):
    """
    Generate a unique filename by appending a counter if the file already exists.
    """
    if not os.path.exists(f"{base_filename}.mp4"):
        return base_filename

    counter = 2
    while os.path.exists(f"{base_filename}_{counter}.mp4"):
        counter += 1
    return f"{base_filename}_{counter}"

def download_video(url, selected_format, output_path="/content/downloaded_video"):
    """
    Download the video using selected format specifications.
    Includes progress tracking and error handling.
    """
    format_id = selected_format['format_id']
    resolution = selected_format['resolution']

    output_file = generate_unique_filename(output_path)

    ydl_opts = {
        'outtmpl': f'{output_file}.%(ext)s',
        'format': f'{format_id}+bestaudio',  # This will automatically merge video and best audio
        'merge_output_format': 'mp4',
        'progress_hooks': [lambda d: print(f"Download Progress: {d.get('_percent_str', 'calculating...')}")],
        'verbose': True
    }

    print(f"\nStarting download of {resolution}p video...")
    print(f"Using format ID: {format_id} with best available audio")

    with YoutubeDL(ydl_opts) as ydl:
        try:
            ydl.download([url])
            print(f"\nDownload successful! File saved as {output_file}.mp4")
            return True
        except Exception as e:
            print(f"\nError during download:")
            print(f"Error message: {str(e)}")
            return False

def validate_youtube_url(url):
    """
    Validate the provided YouTube URL format.
    """
    valid_patterns = [
        'youtube.com/watch?v=',
        'youtu.be/',
        'm.youtube.com/watch?v='
    ]
    return any(pattern in url for pattern in valid_patterns)

def main():
    """
    Main function to orchestrate the video download process with enhanced error handling.
    """
    print("YouTube Video Downloader for Colab")
    print("-" * 40)

    # Get video URL from user
    video_url = input("Enter the YouTube video URL: ").strip()
    if not video_url:
        print("Error: URL cannot be empty")
        return

    # Validate URL format
    if not validate_youtube_url(video_url):
        print("Error: Invalid YouTube URL format")
        print("Please provide a URL in one of these formats:")
        print("- https://www.youtube.com/watch?v=VIDEO_ID")
        print("- https://youtu.be/VIDEO_ID")
        return

    print("\nFetching available video formats...")
    print("This may take a few moments...")
    available_formats = get_available_formats(video_url)

    if not available_formats:
        print("\nError: No suitable formats found. Please check:")
        print("1. The URL is correct and the video exists")
        print("2. The video is not age-restricted or private")
        print("3. YouTube's regional restrictions are not blocking access")
        print("4. Your internet connection is stable")
        return

    # Get and display quality options
    quality_options = get_video_quality_options(available_formats)
    print("\nAvailable video qualities:")

    quality_map = {}
    for i, (resolution, formats) in enumerate(quality_options, start=1):
        format_note = formats[0].get('format_note', '')
        print(f"{i}. {resolution}p {format_note}")
        quality_map[i] = formats[0]

    # Get user's quality selection
    try:
        quality_choice = int(input("\nSelect quality (enter number): "))
        if quality_choice not in quality_map:
            print("Error: Invalid selection")
            return
    except ValueError:
        print("Error: Please enter a valid number")
        return

    # Download the video
    selected_format = quality_map[quality_choice]
    download_video(video_url, selected_format)

if __name__ == "__main__":
    # Ensure yt-dlp is installed and up to date
    try:
        import yt_dlp
    except ImportError:
        print("Installing required package: yt-dlp")
        !pip install -U yt_dlp
        import yt_dlp

    main()

##Extract Audio from Video Downloaded
output to `output_audio.mp3`

In [ ]:
# Import necessary libraries
import subprocess

# Function to extract audio from the downloaded video
def extract_audio(video_path, output_audio_path="output_audio.mp3"):
    """
    Extracts audio from a video file and saves it as an MP3 file using ffmpeg.

    Args:
        video_path (str): Path to the downloaded video file.
        output_audio_path (str): Path to save the extracted audio file.
    """
    # Command to extract audio using ffmpeg
    command = [
        'ffmpeg', '-i', video_path,  # Input video file
        '-vn',  # Disable video (only extract audio)
        '-acodec', 'mp3',  # Set audio codec to mp3
        '-ab', '192k',  # Set audio bitrate
        output_audio_path  # Output audio file path
    ]

    # Execute the ffmpeg command
    subprocess.run(command, check=True)
    print(f"Audio extracted and saved as: {output_audio_path}")

# Example Usage
# Assuming the video file is already downloaded as 'downloaded_video.mp4'
video_path = 'downloaded_video.mp4'  # Path to the existing video file
extract_audio(video_path, 'output_audio.mp3')  # Extract audio and save as mp3


##Download Files to Local Disk
*   Select individual video file to download
<br> **or** <br>
*   Download all detected video files



In [ ]:
import os
from google.colab import files

# Function to list all files in the /content/ directory
def list_downloaded_files(directory="/content/"):
    # List all files in the directory
    files_in_directory = os.listdir(directory)
    # Filter only mp4 files (or other video formats you are interested in)
    downloaded_files = [f for f in files_in_directory if f.endswith('.mp4')]
    return downloaded_files

# Function to download a selected file
def download_selected_file(file_name, directory="/content/"):
    file_path = os.path.join(directory, file_name)
    if os.path.exists(file_path):
        files.download(file_path)
        print(f"Download initiated for {file_name}")
    else:
        print("File does not exist.")

# List all downloaded files
downloaded_files = list_downloaded_files()

# Check if there are any files available
if downloaded_files:
    print("Downloaded files in /content/:")
    for idx, file in enumerate(downloaded_files, 1):
        print(f"{idx}. {file}")

    # Ask the user to select a file
    try:
        user_choice = int(input("Enter the number of the file you want to download: "))
        if 1 <= user_choice <= len(downloaded_files):
            selected_file = downloaded_files[user_choice - 1]
            confirm = input(f"Are you sure you want to download {selected_file}? (y/n): ").strip().lower()

            if confirm == 'y':
                # Download the selected file
                download_selected_file(selected_file)
            else:
                print("Download cancelled.")
        else:
            print("Invalid choice. Exiting.")
    except ValueError:
        print("Invalid input. Exiting.")
else:
    print("No files found in /content/ to download.")


##WIP

### Convert downloaded video to other qualities etc

In [ ]:
## Convert audio to text based on openai whisper module?

In [ ]:
#Install required dependancies
!pip install git+https://github.com/openai/whisper.git
!pip install torch
!pip install whisper


In [ ]:
import whisper
import torch
import logging
from pathlib import Path
import time

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

def transcribe_audio(audio_path, output_path=None, model_size="base"):
    """
    Transcribe audio file using OpenAI's Whisper model.

    Args:
        audio_path (str): Path to the input audio file
        output_path (str, optional): Path to save the transcription
        model_size (str): Size of the Whisper model to use
                         (tiny, base, small, medium, large)

    Returns:
        str: Transcribed text
    """
    try:
        # Load the Whisper model
        logging.info(f"Loading Whisper model: {model_size}")
        model = whisper.load_model(model_size)

        # Check if CUDA is available
        device = "cuda" if torch.cuda.is_available() else "cpu"
        logging.info(f"Using device: {device}")

        # Start transcription
        start_time = time.time()
        logging.info(f"Starting transcription of: {audio_path}")

        # Perform transcription
        result = model.transcribe(audio_path)
        transcribed_text = result["text"]

        # Calculate transcription time
        transcription_time = time.time() - start_time
        logging.info(f"Transcription completed in {transcription_time:.2f} seconds")

        # Save transcription if output path is provided
        if output_path:
            output_path = Path(output_path)
            output_path.write_text(transcribed_text, encoding='utf-8')
            logging.info(f"Transcription saved to: {output_path}")

        return transcribed_text

    except Exception as e:
        logging.error(f"An error occurred during transcription: {str(e)}")
        raise

def main():
    # Define file paths
    input_video = "downloaded_video.mp4"
    output_audio = "output_audio.mp3"
    transcription_output = "transcription.txt"

    try:
        # Transcribe the audio file
        transcribed_text = transcribe_audio(
            audio_path=output_audio,
            output_path=transcription_output,
            model_size="base"  # You can change this to tiny, small, medium, or large
        )

        print("\nTranscription Preview:")
        print("-" * 50)
        print(transcribed_text[:500] + "..." if len(transcribed_text) > 500 else transcribed_text)

    except Exception as e:
        logging.error(f"Failed to process audio: {str(e)}")
        raise

if __name__ == "__main__":
    main()

### Batch Download from url with multiple **videos**

### Batch download via multiple unique url inputs